In [1]:
import numpy as np
from numpy.typing import NDArray
from supporters import *
import SimpleITK as sitk
import nibabel as nib
import os
import stl
import skimage

Loading volumetric data and segmentation

In [2]:
img_path = "D:\Documents\GitHub\VascuIAR\DeepLearning\data\VnRawData\VHSCDD_raw_data\VHSCDD_020_image\ct_020_image.nii.gz"
img_raw = sitk.ReadImage(img_path, sitk.sitkFloat32)
img = sitk.GetArrayFromImage(img_raw)

seg_path = "D:\Documents\GitHub\VascuIAR\DeepLearning\data\VnRawData\VHSCDD_sep_labels\VHSCDD_020_label\ct_020_label_12.nii.gz"
seg_raw = sitk.ReadImage(seg_path, sitk.sitkFloat32)
seg = sitk.GetArrayFromImage(seg_raw)

Implementation of Marching Cubes

In [11]:
DirectionX = 0
DirectionY = 1
DirectionZ = 2

EDGE_DELTA: NDArray[np.uint8]
EDGE_DELTA = np.asarray(
    [
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [0, 0, 1],
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
    ],
    dtype=np.uint8,
)

EDGE_DIRECTION: NDArray[np.uint8]
EDGE_DIRECTION = np.asarray(
    [
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        2,
        2,
        2,
        2,
    ],
    dtype=np.uint8,
)

GEOMETRY_LOOKUP: NDArray[np.int8]
GEOMETRY_LOOKUP = np.asarray(
    [
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 0 = 00000000
        [0, 8, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 1 = 00000001
        [0, 1, 9, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 2 = 00000010
        [8, 3, 9, 1, 3, 9, -1, -1, -1, -1, -1, -1],  # 3 = 00000011
        [1, 2, 10, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 4 = 00000100
        [0, 8, 3, 1, 2, 10, -1, -1, -1, -1, -1, -1],  # 5 = 00000101
        [9, 10, 0, 2, 10, 0, -1, -1, -1, -1, -1, -1],  # 6 = 00000110
        [8, 3, 10, 2, 3, 10, 10, 9, 8, -1, -1, -1],  # 7 = 00000111
        [3, 11, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 8 = 00001000
        [0, 2, 8, 11, 2, 8, -1, -1, -1, -1, -1, -1],  # 9 = 00001001
        [1, 9, 0, 2, 3, 11, -1, -1, -1, -1, -1, -1],  # 10 = 00001010
        [1, 2, 9, 11, 2, 9, 9, 8, 11, -1, -1, -1],  # 11 = 00001011
        [10, 1, 11, 3, 1, 11, -1, -1, -1, -1, -1, -1],  # 12 = 00001100
        [0, 1, 8, 10, 1, 8, 8, 11, 10, -1, -1, -1],  # 13 = 00001101
        [9, 0, 11, 3, 0, 11, 11, 10, 9, -1, -1, -1],  # 14 = 00001110
        [8, 9, 11, 10, 9, 11, -1, -1, -1, -1, -1, -1],  # 15 = 00001111
        [4, 7, 8, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 16 = 00010000
        [3, 0, 7, 4, 0, 7, -1, -1, -1, -1, -1, -1],  # 17 = 00010001
        [0, 1, 9, 8, 4, 7, -1, -1, -1, -1, -1, -1],  # 18 = 00010010
        [1, 9, 7, 4, 9, 7, 7, 3, 1, -1, -1, -1],  # 19 = 00010011
        [1, 2, 10, 8, 4, 7, -1, -1, -1, -1, -1, -1],  # 20 = 00010100
        [1, 2, 10, 0, 4, 3, 7, 4, 3, -1, -1, -1],  # 21 = 00010101
        [8, 4, 7, 0, 2, 9, 10, 2, 9, -1, -1, -1],  # 22 = 00010110
        [3, 2, 10, 3, 10, 4, 3, 4, 7, 4, 10, 9],  # 23 = 00010111
        [8, 4, 7, 3, 11, 2, -1, -1, -1, -1, -1, -1],  # 24 = 00011000
        [11, 7, 2, 4, 7, 2, 2, 0, 4, -1, -1, -1],  # 25 = 00011001
        [8, 4, 7, 9, 0, 1, 11, 2, 3, -1, -1, -1],  # 26 = 00011010
        [4, 7, 11, 9, 4, 2, 11, 4, 2, 9, 2, 1],  # 27 = 00011011
        [4, 7, 8, 1, 3, 10, 11, 10, 3, -1, -1, -1],  # 28 = 00011100
        [10, 7, 11, 10, 0, 7, 10, 1, 0, 4, 7, 0],  # 29 = 00011101
        [4, 7, 8, 11, 0, 3, 0, 11, 9, 10, 9, 11],  # 30 = 00011110
        [11, 7, 9, 4, 7, 9, 9, 11, 10, -1, -1, -1],  # 31 = 00011111
        [9, 5, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 32 = 00100000
        [9, 5, 4, 0, 8, 3, -1, -1, -1, -1, -1, -1],  # 33 = 00100001
        [0, 4, 1, 5, 4, 1, -1, -1, -1, -1, -1, -1],  # 34 = 00100010
        [8, 4, 3, 5, 4, 3, 3, 1, 5, -1, -1, -1],  # 35 = 00100011
        [1, 2, 10, 9, 5, 4, -1, -1, -1, -1, -1, -1],  # 36 = 00100100
        [9, 5, 4, 10, 1, 2, 8, 3, 0, -1, -1, -1],  # 37 = 00100101
        [2, 10, 4, 5, 10, 4, 4, 0, 2, -1, -1, -1],  # 38 = 00100110
        [3, 2, 10, 8, 3, 10, 8, 10, 5, 8, 5, 4],  # 39 = 00100111
        [9, 5, 4, 2, 3, 11, -1, -1, -1, -1, -1, -1],  # 40 = 00101000
        [5, 4, 9, 0, 2, 8, 11, 2, 8, -1, -1, -1],  # 41 = 00101001
        [2, 3, 11, 4, 1, 5, 1, 4, 0, -1, -1, -1],  # 42 = 00101010
        [11, 2, 1, 11, 1, 4, 11, 4, 8, 5, 4, 1],  # 43 = 00101011
        [9, 5, 4, 11, 1, 3, 1, 11, 10, -1, -1, -1],  # 44 = 00101100
        [5, 4, 9, 8, 0, 10, 1, 0, 10, 11, 10, 8],  # 45 = 00101101
        [5, 4, 0, 5, 0, 11, 5, 11, 10, 11, 0, 3],  # 46 = 00101110
        [8, 4, 10, 5, 4, 10, 10, 8, 11, -1, -1, -1],  # 47 = 00101111
        [9, 8, 5, 7, 8, 5, -1, -1, -1, -1, -1, -1],  # 48 = 00110000
        [9, 0, 5, 3, 0, 5, 5, 7, 3, -1, -1, -1],  # 49 = 00110001
        [0, 8, 1, 7, 8, 1, 1, 5, 7, -1, -1, -1],  # 50 = 00110010
        [1, 5, 3, 3, 5, 7, -1, -1, -1, -1, -1, -1],  # 51 = 00110011
        [10, 1, 2, 8, 5, 7, 5, 8, 9, -1, -1, -1],  # 52 = 00110100
        [1, 2, 10, 9, 0, 5, 3, 0, 5, 3, 5, 7],  # 53 = 00110101
        [8, 0, 2, 8, 2, 5, 8, 5, 7, 10, 5, 2],  # 54 = 00110110
        [2, 10, 3, 5, 10, 3, 3, 5, 7, -1, -1, -1],  # 55 = 00110111
        [2, 3, 11, 5, 7, 9, 8, 9, 7, -1, -1, -1],  # 56 = 00111000
        [0, 9, 5, 0, 5, 11, 0, 11, 2, 11, 5, 7],  # 57 = 00111001
        [11, 2, 3, 8, 0, 7, 1, 0, 7, 5, 7, 1],  # 58 = 00111010
        [1, 2, 7, 11, 2, 7, 7, 1, 5, -1, -1, -1],  # 59 = 00111011
        [7, 9, 5, 9, 7, 8, 10, 1, 11, 3, 11, 1],  # 60 = 00111100
        [9, 1, 0, 11, 7, 10, 5, 7, 10, -1, -1, -1],  # 61 = 00111101
        [0, 3, 8, 10, 5, 11, 7, 5, 11, -1, -1, -1],  # 62 = 00111110
        [11, 10, 7, 5, 10, 7, -1, -1, -1, -1, -1, -1],  # 63 = 00111111
        [10, 6, 5, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 64 = 01000000
        [0, 8, 3, 5, 10, 6, -1, -1, -1, -1, -1, -1],  # 65 = 01000001
        [9, 0, 1, 5, 10, 6, -1, -1, -1, -1, -1, -1],  # 66 = 01000010
        [6, 5, 10, 8, 3, 9, 1, 3, 9, -1, -1, -1],  # 67 = 01000011
        [1, 5, 2, 6, 5, 2, -1, -1, -1, -1, -1, -1],  # 68 = 01000100
        [3, 0, 8, 2, 6, 1, 5, 6, 1, -1, -1, -1],  # 69 = 01000101
        [9, 5, 0, 6, 5, 0, 0, 2, 6, -1, -1, -1],  # 70 = 01000110
        [5, 9, 8, 5, 8, 2, 5, 2, 6, 3, 2, 8],  # 71 = 01000111
        [2, 3, 11, 10, 6, 5, -1, -1, -1, -1, -1, -1],  # 72 = 01001000
        [10, 6, 5, 8, 0, 11, 2, 0, 11, -1, -1, -1],  # 73 = 01001001
        [10, 6, 5, 11, 2, 3, 9, 0, 1, -1, -1, -1],  # 74 = 01001010
        [6, 5, 10, 9, 2, 1, 2, 9, 11, 8, 11, 9],  # 75 = 01001011
        [3, 11, 5, 6, 11, 5, 5, 1, 3, -1, -1, -1],  # 76 = 01001100
        [0, 8, 11, 0, 11, 5, 0, 5, 1, 5, 11, 6],  # 77 = 01001101
        [11, 6, 5, 3, 11, 5, 3, 5, 9, 3, 9, 0],  # 78 = 01001110
        [9, 5, 11, 6, 5, 11, 11, 9, 8, -1, -1, -1],  # 79 = 01001111
        [5, 10, 6, 4, 7, 8, -1, -1, -1, -1, -1, -1],  # 80 = 01010000
        [10, 6, 5, 3, 0, 7, 4, 0, 7, -1, -1, -1],  # 81 = 01010001
        [0, 1, 9, 6, 5, 10, 4, 7, 8, -1, -1, -1],  # 82 = 01010010
        [10, 6, 5, 9, 4, 1, 7, 4, 1, 3, 1, 7],  # 83 = 01010011
        [8, 4, 7, 1, 2, 5, 6, 2, 5, -1, -1, -1],  # 84 = 01010100
        [2, 1, 6, 5, 1, 6, 3, 0, 7, 4, 0, 7],  # 85 = 01010101
        [4, 7, 8, 9, 5, 0, 6, 5, 0, 6, 0, 2],  # 86 = 01010110
        [4, 5, 9, 2, 7, 3, 7, 2, 6, -1, -1, -1],  # 87 = 01010111
        [2, 3, 11, 4, 7, 8, 6, 5, 10, -1, -1, -1],  # 88 = 01011000
        [6, 5, 10, 7, 11, 4, 2, 4, 11, 4, 2, 0],  # 89 = 01011001
        [8, 4, 7, 2, 3, 11, 9, 0, 1, 10, 6, 5],  # 90 = 01011010
        [10, 2, 1, 11, 6, 7, 9, 4, 5, -1, -1, -1],  # 91 = 01011011
        [8, 4, 7, 11, 6, 3, 5, 6, 3, 1, 3, 5],  # 92 = 01011100
        [6, 7, 11, 0, 5, 1, 5, 0, 4, -1, -1, -1],  # 93 = 01011101
        [8, 0, 3, 9, 4, 5, 11, 6, 7, -1, -1, -1],  # 94 = 01011110
        [5, 9, 4, 11, 6, 7, -1, -1, -1, -1, -1, -1],  # 95 = 01011111
        [10, 9, 6, 4, 9, 6, -1, -1, -1, -1, -1, -1],  # 96 = 01100000
        [3, 0, 8, 10, 6, 9, 4, 6, 9, -1, -1, -1],  # 97 = 01100001
        [0, 1, 6, 10, 1, 6, 6, 4, 0, -1, -1, -1],  # 98 = 01100010
        [8, 3, 1, 8, 1, 6, 8, 6, 4, 6, 1, 10],  # 99 = 01100011
        [1, 9, 2, 4, 9, 2, 2, 6, 4, -1, -1, -1],  # 100 = 01100100
        [8, 3, 0, 2, 9, 1, 9, 2, 4, 6, 4, 2],  # 101 = 01100101
        [2, 0, 6, 4, 0, 6, -1, -1, -1, -1, -1, -1],  # 102 = 01100110
        [8, 3, 4, 2, 3, 4, 4, 2, 6, -1, -1, -1],  # 103 = 01100111
        [11, 2, 3, 9, 4, 10, 6, 4, 10, -1, -1, -1],  # 104 = 01101000
        [0, 8, 2, 11, 8, 2, 9, 4, 10, 6, 4, 10],  # 105 = 01101001
        [2, 3, 11, 0, 1, 6, 10, 1, 6, 0, 6, 4],  # 106 = 01101010
        [10, 2, 1, 8, 4, 11, 6, 4, 11, -1, -1, -1],  # 107 = 01101011
        [1, 4, 9, 1, 11, 4, 1, 3, 11, 6, 4, 11],  # 108 = 01101100
        [1, 0, 9, 11, 6, 8, 4, 6, 8, -1, -1, -1],  # 109 = 01101101
        [3, 11, 0, 6, 11, 0, 0, 6, 4, -1, -1, -1],  # 110 = 01101110
        [8, 4, 11, 6, 4, 11, -1, -1, -1, -1, -1, -1],  # 111 = 01101111
        [10, 6, 8, 7, 6, 8, 8, 9, 10, -1, -1, -1],  # 112 = 01110000
        [0, 7, 3, 0, 10, 7, 0, 9, 10, 6, 7, 10],  # 113 = 01110001
        [1, 10, 6, 0, 1, 7, 6, 1, 7, 0, 7, 8],  # 114 = 01110010
        [10, 6, 1, 7, 6, 1, 1, 7, 3, -1, -1, -1],  # 115 = 01110011
        [9, 1, 2, 9, 2, 7, 9, 7, 8, 7, 2, 6],  # 116 = 01110100
        [9, 1, 0, 7, 3, 6, 2, 6, 3, -1, -1, -1],  # 117 = 01110101
        [0, 8, 6, 7, 8, 6, 6, 0, 2, -1, -1, -1],  # 118 = 01110110
        [2, 3, 6, 7, 3, 6, -1, -1, -1, -1, -1, -1],  # 119 = 01110111
        [11, 2, 3, 10, 6, 8, 7, 6, 8, 10, 8, 9],  # 120 = 01111000
        [11, 6, 7, 9, 0, 10, 2, 0, 10, -1, -1, -1],  # 121 = 01111001
        [6, 7, 11, 0, 3, 8, 2, 1, 10, -1, -1, -1],  # 122 = 01111010
        [2, 1, 10, 7, 11, 6, -1, -1, -1, -1, -1, -1],  # 123 = 01111011
        [6, 7, 11, 9, 1, 8, 3, 1, 8, -1, -1, -1],  # 124 = 01111100
        [9, 1, 0, 6, 7, 11, -1, -1, -1, -1, -1, -1],  # 125 = 01111101
        [8, 0, 3, 6, 7, 11, -1, -1, -1, -1, -1, -1],  # 126 = 01111110
        [7, 11, 6, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 127 = 01111111
        [7, 6, 11, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 128 = 10000000
        [3, 0, 8, 11, 7, 6, -1, -1, -1, -1, -1, -1],  # 129 = 10000001
        [0, 1, 9, 11, 7, 6, -1, -1, -1, -1, -1, -1],  # 130 = 10000010
        [11, 7, 6, 9, 1, 8, 3, 1, 8, -1, -1, -1],  # 131 = 10000011
        [10, 1, 2, 6, 11, 7, -1, -1, -1, -1, -1, -1],  # 132 = 10000100
        [11, 7, 6, 8, 3, 0, 10, 1, 2, -1, -1, -1],  # 133 = 10000101
        [7, 6, 11, 9, 0, 10, 2, 0, 10, -1, -1, -1],  # 134 = 10000110
        [7, 6, 11, 10, 2, 8, 3, 2, 8, 9, 8, 10],  # 135 = 10000111
        [2, 3, 6, 7, 3, 6, -1, -1, -1, -1, -1, -1],  # 136 = 10001000
        [0, 8, 6, 7, 8, 6, 6, 2, 0, -1, -1, -1],  # 137 = 10001001
        [0, 1, 9, 6, 3, 7, 3, 6, 2, -1, -1, -1],  # 138 = 10001010
        [9, 2, 1, 9, 7, 2, 9, 8, 7, 7, 6, 2],  # 139 = 10001011
        [10, 6, 1, 7, 6, 1, 1, 3, 7, -1, -1, -1],  # 140 = 10001100
        [1, 6, 10, 0, 1, 7, 6, 1, 7, 0, 8, 7],  # 141 = 10001101
        [0, 3, 7, 0, 7, 10, 0, 10, 9, 6, 10, 7],  # 142 = 10001110
        [10, 6, 8, 7, 6, 8, 8, 10, 9, -1, -1, -1],  # 143 = 10001111
        [8, 4, 11, 6, 4, 11, -1, -1, -1, -1, -1, -1],  # 144 = 10010000
        [3, 11, 0, 6, 11, 0, 0, 4, 6, -1, -1, -1],  # 145 = 10010001
        [9, 0, 1, 11, 6, 8, 4, 6, 8, -1, -1, -1],  # 146 = 10010010
        [1, 9, 4, 1, 4, 11, 1, 11, 3, 6, 11, 4],  # 147 = 10010011
        [1, 2, 10, 8, 4, 11, 6, 4, 11, -1, -1, -1],  # 148 = 10010100
        [10, 1, 2, 0, 3, 6, 11, 3, 6, 4, 6, 0],  # 149 = 10010101
        [9, 0, 10, 2, 10, 0, 6, 8, 4, 8, 6, 11],  # 150 = 10010110
        [3, 2, 11, 9, 4, 10, 6, 4, 10, -1, -1, -1],  # 151 = 10010111
        [8, 3, 4, 2, 3, 4, 4, 6, 2, -1, -1, -1],  # 152 = 10011000
        [2, 0, 6, 4, 0, 6, -1, -1, -1, -1, -1, -1],  # 153 = 10011001
        [0, 1, 9, 3, 8, 2, 4, 2, 8, 2, 4, 6],  # 154 = 10011010
        [1, 9, 2, 4, 9, 2, 2, 4, 6, -1, -1, -1],  # 155 = 10011011
        [8, 1, 3, 8, 6, 1, 8, 4, 6, 6, 10, 1],  # 156 = 10011100
        [0, 1, 6, 10, 1, 6, 6, 0, 4, -1, -1, -1],  # 157 = 10011101
        [8, 0, 3, 10, 6, 9, 4, 6, 9, -1, -1, -1],  # 158 = 10011110
        [10, 9, 6, 4, 9, 6, -1, -1, -1, -1, -1, -1],  # 159 = 10011111
        [4, 9, 5, 7, 6, 11, -1, -1, -1, -1, -1, -1],  # 160 = 10100000
        [3, 0, 8, 5, 4, 9, 7, 6, 11, -1, -1, -1],  # 161 = 10100001
        [11, 7, 6, 1, 5, 0, 4, 0, 5, -1, -1, -1],  # 162 = 10100010
        [7, 6, 11, 8, 4, 3, 5, 4, 3, 5, 3, 1],  # 163 = 10100011
        [1, 2, 10, 7, 6, 11, 5, 4, 9, -1, -1, -1],  # 164 = 10100100
        [11, 7, 6, 1, 2, 10, 8, 3, 0, 9, 5, 4],  # 165 = 10100101
        [11, 7, 6, 4, 10, 5, 10, 4, 2, 0, 2, 4],  # 166 = 10100110
        [11, 3, 2, 8, 7, 4, 10, 5, 6, -1, -1, -1],  # 167 = 10100111
        [9, 5, 4, 3, 7, 2, 6, 2, 7, -1, -1, -1],  # 168 = 10101000
        [9, 5, 4, 8, 7, 0, 6, 7, 0, 2, 0, 6],  # 169 = 10101001
        [7, 2, 3, 2, 7, 6, 0, 1, 4, 5, 4, 1],  # 170 = 10101010
        [7, 4, 8, 1, 2, 5, 6, 2, 5, -1, -1, -1],  # 171 = 10101011
        [5, 4, 9, 10, 6, 1, 7, 6, 1, 7, 1, 3],  # 172 = 10101100
        [9, 1, 0, 10, 5, 6, 8, 7, 4, -1, -1, -1],  # 173 = 10101101
        [5, 6, 10, 3, 0, 7, 4, 0, 7, -1, -1, -1],  # 174 = 10101110
        [6, 10, 5, 8, 7, 4, -1, -1, -1, -1, -1, -1],  # 175 = 10101111
        [9, 5, 11, 6, 5, 11, 11, 8, 9, -1, -1, -1],  # 176 = 10110000
        [11, 5, 6, 3, 5, 11, 3, 9, 5, 3, 0, 9],  # 177 = 10110001
        [0, 11, 8, 0, 5, 11, 0, 1, 5, 5, 6, 11],  # 178 = 10110010
        [3, 11, 5, 6, 11, 5, 5, 3, 1, -1, -1, -1],  # 179 = 10110011
        [10, 1, 2, 5, 6, 9, 11, 9, 6, 9, 11, 8],  # 180 = 10110100
        [5, 6, 10, 3, 2, 11, 1, 0, 9, -1, -1, -1],  # 181 = 10110101
        [5, 6, 10, 8, 0, 11, 2, 0, 11, -1, -1, -1],  # 182 = 10110110
        [11, 3, 2, 5, 6, 10, -1, -1, -1, -1, -1, -1],  # 183 = 10110111
        [5, 8, 9, 5, 2, 8, 5, 6, 2, 3, 8, 2],  # 184 = 10111000
        [9, 5, 0, 6, 5, 0, 0, 6, 2, -1, -1, -1],  # 185 = 10111001
        [8, 0, 3, 2, 6, 1, 5, 6, 1, -1, -1, -1],  # 186 = 10111010
        [1, 5, 2, 6, 5, 2, -1, -1, -1, -1, -1, -1],  # 187 = 10111011
        [10, 5, 6, 8, 3, 9, 1, 3, 9, -1, -1, -1],  # 188 = 10111100
        [1, 0, 9, 6, 10, 5, -1, -1, -1, -1, -1, -1],  # 189 = 10111101
        [3, 8, 0, 6, 10, 5, -1, -1, -1, -1, -1, -1],  # 190 = 10111110
        [10, 5, 6, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 191 = 10111111
        [11, 10, 7, 5, 10, 7, -1, -1, -1, -1, -1, -1],  # 192 = 11000000
        [8, 3, 0, 10, 5, 11, 7, 5, 11, -1, -1, -1],  # 193 = 11000001
        [0, 1, 9, 11, 7, 10, 5, 7, 10, -1, -1, -1],  # 194 = 11000010
        [10, 5, 11, 7, 5, 11, 8, 9, 3, 1, 9, 3],  # 195 = 11000011
        [1, 2, 7, 11, 2, 7, 7, 5, 1, -1, -1, -1],  # 196 = 11000100
        [3, 0, 8, 1, 2, 7, 11, 2, 7, 1, 7, 5],  # 197 = 11000101
        [0, 5, 9, 0, 11, 5, 0, 2, 11, 11, 7, 5],  # 198 = 11000110
        [11, 3, 2, 9, 7, 5, 7, 9, 8, -1, -1, -1],  # 199 = 11000111
        [2, 10, 3, 5, 10, 3, 3, 7, 5, -1, -1, -1],  # 200 = 11001000
        [8, 2, 0, 8, 5, 2, 8, 7, 5, 10, 2, 5],  # 201 = 11001001
        [9, 0, 1, 10, 2, 5, 3, 2, 5, 7, 5, 3],  # 202 = 11001010
        [2, 1, 10, 7, 5, 8, 9, 8, 5, -1, -1, -1],  # 203 = 11001011
        [1, 3, 5, 3, 7, 5, -1, -1, -1, -1, -1, -1],  # 204 = 11001100
        [0, 8, 1, 7, 8, 1, 1, 7, 5, -1, -1, -1],  # 205 = 11001101
        [9, 0, 5, 3, 0, 5, 5, 3, 7, -1, -1, -1],  # 206 = 11001110
        [9, 8, 5, 7, 8, 5, -1, -1, -1, -1, -1, -1],  # 207 = 11001111
        [8, 4, 10, 5, 4, 10, 10, 11, 8, -1, -1, -1],  # 208 = 11010000
        [5, 0, 4, 5, 11, 0, 5, 10, 11, 11, 3, 0],  # 209 = 11010001
        [9, 0, 1, 8, 4, 10, 5, 4, 10, 8, 10, 11],  # 210 = 11010010
        [4, 5, 9, 3, 1, 11, 10, 11, 1, -1, -1, -1],  # 211 = 11010011
        [11, 1, 2, 11, 4, 1, 11, 8, 4, 5, 1, 4],  # 212 = 11010100
        [11, 3, 2, 5, 1, 4, 0, 4, 1, -1, -1, -1],  # 213 = 11010101
        [9, 4, 5, 0, 2, 8, 11, 2, 8, -1, -1, -1],  # 214 = 11010110
        [4, 5, 9, 11, 3, 2, -1, -1, -1, -1, -1, -1],  # 215 = 11010111
        [3, 10, 2, 8, 10, 3, 8, 5, 10, 8, 4, 5],  # 216 = 11011000
        [2, 10, 4, 5, 10, 4, 4, 2, 0, -1, -1, -1],  # 217 = 11011001
        [4, 5, 9, 2, 1, 10, 0, 3, 8, -1, -1, -1],  # 218 = 11011010
        [10, 2, 1, 4, 5, 9, -1, -1, -1, -1, -1, -1],  # 219 = 11011011
        [8, 4, 3, 5, 4, 3, 3, 5, 1, -1, -1, -1],  # 220 = 11011100
        [0, 4, 1, 5, 4, 1, -1, -1, -1, -1, -1, -1],  # 221 = 11011101
        [4, 5, 9, 3, 8, 0, -1, -1, -1, -1, -1, -1],  # 222 = 11011110
        [9, 4, 5, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 223 = 11011111
        [11, 7, 9, 4, 7, 9, 9, 10, 11, -1, -1, -1],  # 224 = 11100000
        [8, 3, 0, 7, 4, 11, 9, 11, 4, 11, 9, 10],  # 225 = 11100001
        [10, 11, 7, 10, 7, 0, 10, 0, 1, 4, 0, 7],  # 226 = 11100010
        [8, 7, 4, 10, 3, 1, 3, 10, 11, -1, -1, -1],  # 227 = 11100011
        [4, 11, 7, 9, 4, 2, 11, 4, 2, 9, 1, 2],  # 228 = 11100100
        [7, 4, 8, 1, 0, 9, 3, 2, 11, -1, -1, -1],  # 229 = 11100101
        [11, 7, 2, 4, 7, 2, 2, 4, 0, -1, -1, -1],  # 230 = 11100110
        [7, 4, 8, 2, 11, 3, -1, -1, -1, -1, -1, -1],  # 231 = 11100111
        [3, 10, 2, 3, 4, 10, 3, 7, 4, 4, 9, 10],  # 232 = 11101000
        [7, 4, 8, 0, 2, 9, 10, 2, 9, -1, -1, -1],  # 233 = 11101001
        [10, 2, 1, 0, 4, 3, 7, 4, 3, -1, -1, -1],  # 234 = 11101010
        [10, 2, 1, 7, 4, 8, -1, -1, -1, -1, -1, -1],  # 235 = 11101011
        [1, 9, 7, 4, 9, 7, 7, 1, 3, -1, -1, -1],  # 236 = 11101100
        [9, 1, 0, 7, 4, 8, -1, -1, -1, -1, -1, -1],  # 237 = 11101101
        [3, 0, 7, 4, 0, 7, -1, -1, -1, -1, -1, -1],  # 238 = 11101110
        [4, 8, 7, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 239 = 11101111
        [8, 9, 11, 10, 9, 11, -1, -1, -1, -1, -1, -1],  # 240 = 11110000
        [9, 0, 11, 3, 0, 11, 11, 9, 10, -1, -1, -1],  # 241 = 11110001
        [0, 1, 8, 10, 1, 8, 8, 10, 11, -1, -1, -1],  # 242 = 11110010
        [10, 1, 11, 3, 1, 11, -1, -1, -1, -1, -1, -1],  # 243 = 11110011
        [1, 2, 9, 11, 2, 9, 9, 11, 8, -1, -1, -1],  # 244 = 11110100
        [0, 9, 1, 11, 3, 2, -1, -1, -1, -1, -1, -1],  # 245 = 11110101
        [0, 2, 8, 11, 2, 8, -1, -1, -1, -1, -1, -1],  # 246 = 11110110
        [3, 2, 11, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 247 = 11110111
        [8, 3, 10, 2, 3, 10, 10, 8, 9, -1, -1, -1],  # 248 = 11111000
        [9, 10, 0, 2, 10, 0, -1, -1, -1, -1, -1, -1],  # 249 = 11111001
        [3, 8, 0, 10, 2, 1, -1, -1, -1, -1, -1, -1],  # 250 = 11111010
        [1, 10, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 251 = 11111011
        [8, 3, 9, 1, 3, 9, -1, -1, -1, -1, -1, -1],  # 252 = 11111100
        [0, 9, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 253 = 11111101
        [0, 3, 8, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 254 = 11111110
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],  # 255 = 11111111
    ],
    dtype=np.int8,
)

VOLUME_CASE_LOOKUP = [
    0,
    1,     # case 1 # all false
    1/48,  # case 2
    1/8,   # case 3
    1/24,  # case 4
    1/24,  # case 5
    17/48, # case 6
    7/48,  # case 7
    1/16,  # case 8
    1/2,   # case 9
    1/2,   # case 10
    1/4,   # case 11
    (7 + 8**(1/2))/24, # case 12
    139/144, # case 13
    1/12,  # case 14
    5/12,  # case 15
    1-1,   # case 16 # all true --> base case
    1-1/48, # case 17
    1-1/8, # case 18
    1-1/24,  # case 19
    1-1/24,  # case 20
    1-17/48, # case 21
    1-7/48,   # case 22
    1-1/16,   # case 23
    1-1/2,    # case 24
    1-1/2,   # case 25
    1-1/4,   # case 26
    1 - ((7 + 8**(1/2))/24), # case 27
    1-139/144, # case 28
    1-1/12,  # case 29
    1-5/12,  # case 30
]

VOLUME_LOOKUP: NDArray[np.uint8]
VOLUME_LOOKUP = np.asarray(
    [
        1,   # 0 = 00000000
        2,   # 1 = 00000001
        2,   # 2 = 00000010
        3,   # 3 = 00000011
        1,   # 4 = 00000100
        2,   # 5 = 00000101
        3,   # 6 = 00000110
        6,   # 7 = 00000111
        2,   # 8 = 00001000
        2,   # 9 = 00001001
        2,   # 10 = 00001010
        7,   # 11 = 00001011
        3,   # 12 = 00001100
        7,   # 13 = 00001101
        6,   # 14 = 00001110
        9,   # 15 = 00001111
        2,   # 16 = 00010000
        3,   # 17 = 00010001
        4,   # 18 = 00010010
        6,   # 19 = 00010011
        4,   # 20 = 00010100
        7,   # 21 = 00010101
        7,   # 22 = 00010110
        12,  # 23 = 00010111
        4,   # 24 = 00011000
        6,   # 25 = 00011001
        8,   # 26 = 00011010
        15,  # 27 = 00011011
        7,   # 28 = 00011100
        15,  # 29 = 00011101
        13,  # 30 = 00011110
        21,  # 31 = 00011111
        2,   # 32 = 00100000
        4,   # 33 = 00100001
        3,   # 34 = 00100010
        6,   # 35 = 00100011
        4,   # 36 = 00100100
        8,   # 37 = 00100101
        6,   # 38 = 00100110
        10,  # 39 = 00100111
        5,   # 40 = 00101000
        7,   # 41 = 00101001
        7,   # 42 = 00101010
        15,  # 43 = 00101011
        7,   # 44 = 00101100
        13,  # 45 = 00101101
        12,  # 46 = 00101110
        21,  # 47 = 00101111
        3,   # 48 = 00110000
        6,   # 49 = 00110001
        6,   # 50 = 00110010
        9,   # 51 = 00110011
        7,   # 52 = 00110100
        13,  # 53 = 00110101
        15,  # 54 = 00110110
        21,  # 55 = 00110111
        7,   # 56 = 00111000
        12,  # 57 = 00111001
        13,  # 58 = 00111010
        21,  # 59 = 00111011
        11,  # 60 = 00111100
        22,  # 61 = 00111101
        22,  # 62 = 00111110
        18,  # 63 = 00111111
        2,   # 64 = 01000000
        5,   # 65 = 01000001
        4,   # 66 = 01000010
        7,   # 67 = 01000011
        3,   # 68 = 01000100
        7,   # 69 = 01000101
        6,   # 70 = 01000110
        15,  # 71 = 01000111
        4,   # 72 = 01001000
        7,   # 73 = 01001001
        7,   # 74 = 01001010
        15,  # 75 = 01001011
        7,   # 76 = 01001100
        13,  # 77 = 01001101
        12,  # 78 = 01001110
        21,  # 79 = 01001111
        4,   # 80 = 01010000
        7,   # 81 = 01010001
        8,   # 82 = 01010010
        13,  # 83 = 01010011
        3,   # 84 = 01010100
        11,  # 85 = 01010101
        13,  # 86 = 01010110
        7,   # 87 = 01010111
        8,   # 88 = 01011000
        13,  # 89 = 01011001
        14,  # 90 = 01011010
        23,  # 91 = 01011011
        22,  # 92 = 01011100
        20,  # 93 = 01011101
        23,  # 94 = 01011110
        19,  # 95 = 01011111
        3,   # 96 = 01100000
        7,   # 97 = 01100001
        6,   # 98 = 01100010
        12,  # 99 = 01100011
        6,   # 100 = 01100100
        13,  # 101 = 01100101
        9,   # 102 = 01100110
        21,  # 103 = 01100111
        7,   # 104 = 01101000
        11,  # 105 = 01101001
        13,  # 106 = 01101010
        22,  # 107 = 01101011
        15,  # 108 = 01101100
        22,  # 109 = 01101101
        21,  # 110 = 01101110
        18,  # 111 = 01101111
        6,   # 112 = 01110000
        15,  # 113 = 01110001
        10,  # 114 = 01110010
        21,  # 115 = 01110011
        12,  # 116 = 01110100
        20,  # 117 = 01110101
        21,  # 118 = 01110110
        19,  # 119 = 01110111
        13,  # 120 = 01111000
        20,  # 121 = 01111001
        19,  # 122 = 01111010
        19,  # 123 = 01111011
        22,  # 124 = 01111100
        20,  # 125 = 01111101
        19,  # 126 = 01111110
        17,  # 127 = 01111111
        2,   # 128 = 10000000
        4,   # 129 = 10000001
        5,   # 130 = 10000010
        7,   # 131 = 10000011
        4,   # 132 = 10000100
        8,   # 133 = 10000101
        7,  # 134 = 10000110
        13, # 135 = 10000111
        7,  # 136 = 10001000
        6,  # 137 = 10001001
        7,  # 138 = 10001010
        12, # 139 = 10001011
        6,  # 140 = 10001100
        10, # 141 = 10001101
        15, # 142 = 10001110
        21, # 143 = 10001111
        7,  # 144 = 10010000
        6,  # 145 = 10010001
        7,  # 146 = 10010010
        15, # 147 = 10010011
        7,  # 148 = 10010100
        13, # 149 = 10010101
        11, # 150 = 10010110
        22, # 151 = 10010111
        6,  # 152 = 10011000
        9,  # 153 = 10011001
        13, # 154 = 10011010
        21, # 155 = 10011011
        12, # 156 = 10011100
        21, # 157 = 10011101
        22, # 158 = 10011110
        18, # 159 = 10011111
        4,  # 160 = 10100000
        8,  # 161 = 10100001
        7,  # 162 = 10100010
        13, # 163 = 10100011
        8,  # 164 = 10100100
        14, # 165 = 10100101
        13, # 166 = 10100110
        20, # 167 = 10100111
        7,  # 168 = 10101000
        13, # 169 = 10101001
        11, # 170 = 10101010
        7,  # 171 = 10101011
        13, # 172 = 10101100
        23, # 173 = 10101101
        22, # 174 = 10101110
        19, # 175 = 10101111
        6,  # 176 = 10110000
        10, # 177 = 10110001
        12, # 178 = 10110010
        18, # 179 = 10110011
        13, # 180 = 10110100
        19, # 181 = 10110101
        20, # 182 = 10110110
        19, # 183 = 10110111
        15, # 184 = 10111000
        21, # 185 = 10111001
        22, # 186 = 10111010
        18, # 187 = 10111011
        22, # 188 = 10111100
        19, # 189 = 10111101
        20, # 190 = 10111110
        17, # 191 = 10111111
        3,  # 192 = 11000000
        7,  # 193 = 11000001
        7,  # 194 = 11000010
        11, # 195 = 11000011
        6,  # 196 = 11000100
        13, # 197 = 11000101
        12, # 198 = 11000110
        3,  # 199 = 11000111
        6,  # 200 = 11001000
        15, # 201 = 11001001
        13, # 202 = 11001010
        22, # 203 = 11001011
        9,  # 204 = 11001100
        21, # 205 = 11001101
        21, # 206 = 11001110
        18, # 207 = 11001111
        6,  # 208 = 11010000
        12, # 209 = 11010001
        13, # 210 = 11010010
        19, # 211 = 11010011
        15, # 212 = 11010100
        18, # 213 = 11010101
        19, # 214 = 11010110
        20, # 215 = 11010111
        10, # 216 = 11011000
        21, # 217 = 11011001
        23, # 218 = 11011010
        19, # 219 = 11011011
        21, # 220 = 11011100
        18, # 221 = 11011101
        19, # 222 = 11011110
        17, # 223 = 11011111
        6,  # 224 = 11100000
        13, # 225 = 11100001
        15, # 226 = 11100010
        22, # 227 = 11100011
        10, # 228 = 11100100
        23, # 229 = 11100101
        21, # 230 = 11100110
        19, # 231 = 11100111
        12, # 232 = 11101000
        22, # 233 = 11101001
        22, # 234 = 11101010
        20, # 235 = 11101011
        21, # 236 = 11101100
        19, # 237 = 11101101
        18, # 238 = 11101110
        17, # 239 = 11101111
        9,  # 240 = 11110000
        21, # 241 = 11110001
        21, # 242 = 11110010
        18, # 243 = 11110011
        21, # 244 = 11110100
        19, # 245 = 11110101
        18, # 246 = 11110110
        17, # 247 = 11110111
        21, # 248 = 11111000
        18, # 249 = 11111001
        19, # 250 = 11111010
        17, # 251 = 11111011
        18, # 252 = 11111100
        17, # 253 = 11111101
        17, # 254 = 11111110
        16, # 255 = 11111111
    ],
    dtype=np.int8,
)

In [ ]:
def interpolation(a, b, level):
  # zero values to level
  a -= level
  b -= level
  # linear interpolation
  return a / (a - b)

def marching_cubes(volume, level=0.0):
  vertices = list()
  vertex_ids = list()
  triangles = list()
  triangle_ids = list()

  # Compare volume to level
  volume_test = np.asarray(volume >= level, dtype="bool")
  dimX, dimY, dimZ = volume_test.shape

  # Intialize 3D array for FenwickTree
  CUBE_VOLUME = [[[0.0] * (dimZ - 1) for _ in range(dimY - 1)] for _ in range(dimX - 1)]
  CUBE_VOLUME = np.array(CUBE_VOLUME)

  def calculate_vertex_id(x, y, z, direction):
    return (x + y * dimX + z * dimX * dimY) * 3 + direction

  # enumerate volume
  for z in range(dimZ):
    for y in range(dimY):
      for x in range(dimX):

        def edge_to_vertex_id(edge_number):
          dx, dy, dz = EDGE_DELTA[edge_number]
          direction = EDGE_DIRECTION[edge_number]
          return calculate_vertex_id(x + dx, y + dy, z + dz, direction)

        # Find where volume crosses the level --> vertex found
        if x < (dimX-1) and volume_test[x, y, z] !=  volume_test[x + 1, y, z]:
          delta = interpolation(volume[x, y, z], volume[x + 1, y, z], level)
          vertices.append([x + delta, y, z])
          vertex_ids.append(calculate_vertex_id(x, y, z, DirectionX))

        if y < (dimY-1) and volume_test[x, y, z] !=  volume_test[x, y + 1, z]:
          delta = interpolation(volume[x, y, z], volume[x, y + 1, z], level)
          vertices.append([x, y + delta, z])
          vertex_ids.append(calculate_vertex_id(x, y, z, DirectionY))

        if z < (dimZ-1) and volume_test[x, y, z] !=  volume_test[x, y, z + 1]:
          delta = interpolation(volume[x, y, z], volume[x, y, z + 1], level)
          vertices.append([x, y, z + delta])
          vertex_ids.append(calculate_vertex_id(x, y, z, DirectionZ))

        if x == (dimX - 1) or y == (dimY - 1) or z == (dimZ - 1):
          continue

        # Calculate the volume type of a cube
        volume_type = 0
        if volume_test[x, y, z]:
          volume_type |= 1<<0             # 2^0 = 1
        if volume_test[x + 1, y, z]:
          volume_type |= 1<<1             # 2^2 = 2
        if volume_test[x + 1, y + 1, z]:
          volume_type |= 1<<2             # 2^2 = 4
        if volume_test[x, y + 1, z]:
          volume_type |= 1<<3             # 2^3 = 8
        if volume_test[x, y, z + 1]:
          volume_type |= 1<<4             # 2^4 = 16
        if volume_test[x + 1, y, z + 1]:
          volume_type |= 1<<5             # 2^5 = 32
        if volume_test[x + 1, y + 1, z + 1]:
          volume_type |= 1<<6             # 2^6 = 64
        if volume_test[x, y + 1, z + 1]:
          volume_type |= 1<<7             # 2^7 = 128

        CUBE_VOLUME[x,y,z] = VOLUME_CASE_LOOKUP[VOLUME_LOOKUP[volume_type]]

        # By rotaion operation 2^8 = 256 configurations -> 15 configs
        # Lookup Geometry
        lookup = GEOMETRY_LOOKUP[volume_type]
        for i in range(0, len(lookup), 3):
          if lookup[i] < 0:
            break
          edge0, edge1, edge2 = lookup[i : i + 3]
          vertex_id0 = edge_to_vertex_id(edge0)
          vertex_id1 = edge_to_vertex_id(edge1)
          vertex_id2 = edge_to_vertex_id(edge2)
          triangle_ids.append([vertex_id0, vertex_id1, vertex_id2])

  # Convert ids to indexes
  order_of_ids = {id:order for order, id in enumerate(vertex_ids)}
  for triangle_corners in triangle_ids:
    triangles.append([order_of_ids[c] for c in triangle_corners])

  return np.array(vertices), triangles, CUBE_VOLUME

Reconstruction on segmentation result

In [ ]:
verts, faces, cube_volume = marching_cubes(seg, 0.5)

In [ ]:
explore_3D_array(seg)

In [ ]:
filtered_img = img * seg
explore_3D_array(filtered_img)

In [8]:
np.min(filtered_img), np.max(filtered_img), np.average(filtered_img)

(0.0, 1607.0, 3.8660989)

In [ ]:
print(filtered_img)

In [7]:
np.min(img), np.max(img), np.average(img)

(0.0, 2498.0, 911.66846)

In [7]:
np.unique(seg, return_counts=True)

(array([0., 1.], dtype=float32), array([183748409,    571591], dtype=int64))

In [ ]:
import SimpleITK as sitk
import numpy as np
from stl import mesh
from skimage import measure

levels = [1000, 1150, 1100, 1150, 1200, 1250, 1300, 1350]
for level in levels:
    verts, faces, normals, values = measure.marching_cubes(img, level=level)
    obj_3d = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

    for i, f in enumerate(faces):
        obj_3d.vectors[i] = verts[f]

    # Save the STL file with the name and the path
    obj_3d.save(f'whole_{level}.stl')

In [19]:
seg = seg.astype(bool)

In [20]:
np.unique(seg, return_counts=True)

(array([False,  True]), array([183748409,    571591], dtype=int64))

In [22]:
n,m,p = seg.shape           

In [ ]:
for k in range(0, p):
    for i in range(1, n-1):
        for j in range(1, m-1):